# Optimal Power Flow: IEEE 33 Bus Case with Matpower
- Matpower
  - Matpower python 버전 설치: https://pypi.org/project/matpower/
  - Octave 설치: https://github.com/yasirroni/mypower/blob/master/README_INSTALL_OCTAVE_AND_OCT2PY.md
    - 환경변수에 octave-cli.exe 반드시 추가 (참고경로: C:\Program Files\GNU Octave\Octave-10.1.0\mingw64\bin, 홈페이지의 경로와 실제 경로가 다름 주의!)
- Matpower OPF 결과 도출
- Matpower 참고: https://pypi.org/project/matpower/
- IEEE 33 bus 데이터 구조
  - https://github.com/MATPOWER/matpower/blob/master/data/case33bw.m

1. 계통 불러오기

In [14]:
from matpower import start_instance
from oct2py import octave
import pandas as pd
import numpy as np

# Set and load Matpower case
m = start_instance()
mpc = m.loadcase('case33bw')

#Base MVA
base_MVA = mpc['baseMVA']

2. Data 형식
- Bus Data
- Branch Data
- Gen Data
- Load Data
- Y Bus and Connectivity data
- Etc (나중에 고려)

In [15]:
"""
Bus (ref: https://matpower.org/docs/ref/matpower5.0/caseformat.html, http://labs.ece.uw.edu/pstca/formats/cdf.txt, http://labs.ece.uw.edu/pstca/formats/pti.txt)
"""
# 'bus_i': Bus number,
# 'type': Type (0 - Unregulated (load, PQ), 1 - Hold MVAR generation within voltage limits, (PQ), 2 - Hold voltage within VAR limits (gen, PV), 3 - Hold voltage and angle (swing, V-Theta) (must always have one)),
# 'Pd': Load MW[MW],
# 'Qd': Load MVAR[MVar],
# 'Gs': Shunt conductance, MW at 1.0 per unit voltage,
# 'Bs': Shunt susceptance, MVAR at 1.0 per unit voltage. (- = reactor),
# 'area': Area number, 1-100,
# 'Vm': Voltage magnitude, per unit,
# 'Va': Voltage angle, degrees,
# 'baseKV': Base voltage, KV,
# 'zone': Loss zone, 1-999,
# 'Vmax': Maximum voltage limit,
# 'Vmin': Minimum voltage limit.
mat_bus_info_columns = ['bus_i','type','Pd','Qd','Gs','Bs','area','Vm','Va','baseKV','zone','Vmax','Vmin']
mat_bus_info = pd.DataFrame(mpc['bus'], columns = mat_bus_info_columns)

Bus_info = mat_bus_info.copy()
Bus_info['bus_i']=Bus_info['bus_i'].astype(int)
Bus_info['type']=Bus_info['type'].astype(int)

Bus_info.set_index('bus_i',inplace=True)
Bus_info.to_csv('./Pre_cal_data/Bus_info.csv')
Bus_info.head(5)

,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
bus_i,,,,,,,,,,,,
1,3,0.00,0.00,0.0,0.0,1.0,1.0,0.0,12.66,1.0,1.0,1.0
2,1,0.10,0.06,0.0,0.0,1.0,1.0,0.0,12.66,1.0,1.1,0.9
3,1,0.09,0.04,0.0,0.0,1.0,1.0,0.0,12.66,1.0,1.1,0.9
4,1,0.12,0.08,0.0,0.0,1.0,1.0,0.0,12.66,1.0,1.1,0.9
5,1,0.06,0.03,0.0,0.0,1.0,1.0,0.0,12.66,1.0,1.1,0.9


In [16]:
"""
Line (ref: https://matpower.org/docs/ref/matpower5.0/caseformat.html, http://labs.ece.uw.edu/pstca/formats/cdf.txt, http://labs.ece.uw.edu/pstca/formats/pti.txt)
"""
# 'fbus': From bus number,
# 'tbus': To bus number,
# 'r_pu': Resistance, per unit,
# 'x_pu': Reactance, per unit,
# 'b_pu': Total line charging, per unit,
# 'rateA': MVA rating A,
# 'rateB','rateC': Higher MVA ratings,
# 'ratio': Transformer off nominal turns ratio,
# 'angle': Transformer phase shift angle,
# 'status': Initial branch status, 1 - in service, 0 - out of service,
# 'angmin': Minimum phase shift,
# 'angmax': Maximum phase shift
branch_data_column = ['fbus','tbus','r_pu','x_pu','b_pu','rateA','rateB','rateC','ratio','angle','status','angmin','angmax']
branch_data_df = pd.DataFrame(mpc['branch'],columns = branch_data_column)

Line_index = range(1,branch_data_df.shape[0]+1)
Line_column = ['from_bus','to_bus','r_pu','x_pu','b_pu','in_service']
Line_info = pd.DataFrame(index = Line_index, columns = Line_column)

Line_info['from_bus'] = branch_data_df['fbus'].astype(int).values
Line_info['to_bus'] = branch_data_df['tbus'].astype(int).values
Line_info['r_pu'] = branch_data_df['r_pu'].values
Line_info['x_pu'] = branch_data_df['x_pu'].values
Line_info['b_pu'] = branch_data_df['b_pu'].values
Line_info['in_service'] = branch_data_df['status'].astype(int).values

Line_info.index.name = 'Line_l'

Line_info.to_csv('./Pre_cal_data/Line_info.csv')
Line_info.head(5)

,from_bus,to_bus,r_pu,x_pu,b_pu,in_service
Line_l,,,,,,
1,1,2,0.005753,0.002932,0.0,1
2,2,3,0.030760,0.015667,0.0,1
3,3,4,0.022836,0.011630,0.0,1
4,4,5,0.023778,0.012110,0.0,1
5,5,6,0.051099,0.044112,0.0,1


In [17]:
"""
Generator (ref: https://matpower.org/docs/ref/matpower5.0/caseformat.html, http://labs.ece.uw.edu/pstca/formats/cdf.txt, http://labs.ece.uw.edu/pstca/formats/pti.txt)
"""
# 'bus_i': Bus number,
# 'Pg': MW output,
# 'Qg': MVAR output,
# 'Qmax': Max MVAR,
# 'Qmin': Min MVAR,
# 'Vg': voltage magnitude setpoint (p.u.),
# 'mBase': Total MVA base of this machine (or machines), defaults to system MVA base.,
# 'status': Machine status, 1 in service, 0 out of service,
# 'Pmax': Max MW,
# 'Pmin': Min MW,
# 'Pc1': lower real power output of PQ capability curve (MW) ,
# 'Pc2': upper real power output of PQ capability curve (MW),
# 'Qc1min': minimum reactive power output at Pc1 (MVAr),
# 'Qc1max': maximum reactive power output at Pc1 (MVAr),
# 'Qc2min': minimum reactive power output at Pc2 (MVAr),
# 'Qc2max': maximum reactive power output at Pc2 (MVAr),
# 'ramp_agc': ramp rate for load following/AGC (MW/min),
# 'ramp_10': ramp rate for 10 minute reserves (MW),
# 'ramp_30': ramp rate for 30 minute reserves (MW),
# 'ramp_q': ramp rate for reactive power (2 sec timescale) (MVAr/min),	
# 'apf': APF, area participation factor.
mat_gen_info_columns = ['bus','Pg',	'Qg','Qmax','Qmin','Vg','mBase','status','Pmax','Pmin','Pc1','Pc2','Qc1min','Qc1max','Qc2min','Qc2max','ramp_agc','ramp_10','ramp_30','ramp_q',	'apf']
mat_gen_info = pd.DataFrame(mpc['gen'], columns = mat_gen_info_columns)

## Generator Cost Data Format
#%	1	startup	shutdown	n	x1	y1	...	xn	yn
#%	2	startup	shutdown	n	c(n-1)	...	c0
# 'type': model, 1 - piecewise linear, 2 - polynomial,
# 'startup': startup cost in US dollars,
# 'shutdown': shutdown cost in US dollars,
# 'n': number of cost coefficients to follow for polynomial cost function, or number of data points for piecewise linear,
# c(2)~c(0) parameters defining total cost function f(p), units of f and p are $/hr and MW (or MVAr), respectively.
# 이 부분에서는 polynomial 형태로 2차항 계수(c(2)) 부터 상수(c(0))로 가정
mat_gen_cost_info_columns = ['type', 'startup',	'shutdown',	'n','c(2)','c(1)','c(0)']
mat_gen_cost_info = pd.DataFrame(mpc['gencost'], columns = mat_gen_cost_info_columns)

#Gen Data
gen_index = range(1,mat_gen_info.shape[0]+1)
gen_columns = ['bus','in_service','vm_pu','p_mw','max_p_mw','min_p_mw','min_q_mvar','max_q_mvar','c(2)','c(1)','c(0)']
gen_info = pd.DataFrame(index = gen_index, columns = gen_columns)

gen_info['bus']=mat_gen_info['bus'].astype(int).values
gen_info['in_service']=mat_gen_info['status'].astype(int).values
gen_info['vm_pu']=mat_gen_info['Vg'].values
gen_info['p_mw']=mat_gen_info['Pg'].values
gen_info['max_p_mw']=mat_gen_info['Pmax'].values
gen_info['min_p_mw']=mat_gen_info['Pmin'].values
gen_info['min_q_mvar']=mat_gen_info['Qmin'].values
gen_info['max_q_mvar']=mat_gen_info['Qmax'].values
gen_info['c(2)']=mat_gen_cost_info['c(2)'].values
gen_info['c(1)']=mat_gen_cost_info['c(1)'].values
gen_info['c(0)']=mat_gen_cost_info['c(0)'].values


gen_info.to_csv('./Pre_cal_data/Gen_info.csv')
gen_info.head(5)

,bus,in_service,vm_pu,p_mw,max_p_mw,min_p_mw,min_q_mvar,max_q_mvar,c(2),c(1),c(0)
1,1,1,1.0,0.0,10.0,0.0,-10.0,10.0,0.0,20.0,0.0


In [18]:
"""
Load (ref: https://matpower.org/docs/ref/matpower5.0/caseformat.html, http://labs.ece.uw.edu/pstca/formats/cdf.txt, http://labs.ece.uw.edu/pstca/formats/pti.txt)
"""
# 'bus_i': Bus number,
# 'type': Type (0 - Unregulated (load, PQ), 1 - Hold MVAR generation within voltage limits, (PQ), 2 - Hold voltage within VAR limits (gen, PV), 3 - Hold voltage and angle (swing, V-Theta) (must always have one)),
# 'Pd': Load MW[MW],
# 'Qd': Load MVAR[MVar],
# 'Gs': Shunt conductance, MW at 1.0 per unit voltage,
# 'Bs': Shunt susceptance, MVAR at 1.0 per unit voltage. (- = reactor),
# 'area': Area number, 1-100,
# 'Vm': Voltage magnitude, per unit,
# 'Va': Voltage angle, degrees,
# 'baseKV': Base voltage, KV,
# 'zone': Loss zone, 1-999,
# 'Vmax': Maximum voltage limit,
# 'Vmin': Minimum voltage limit.

mat_bus_info_columns = ['bus_i','type','Pd','Qd','Gs','Bs','area','Vm','Va','baseKV','zone','Vmax','Vmin']
mat_bus_info = pd.DataFrame(mpc['bus'], columns = mat_bus_info_columns)

Load_index = range(1,mat_bus_info.shape[0]+1)
Load_column = ['bus','p_mw','q_mvar']
Load_info = pd.DataFrame(index=Load_index, columns = Load_column)

Load_info['bus'] = mat_bus_info['bus_i'].astype(int).values
Load_info['p_mw'] = mat_bus_info['Pd'].values
Load_info['q_mvar'] = mat_bus_info['Qd'].values

Load_info.index.name = 'Load_d'

Load_info.to_csv('./Pre_cal_data/Load_info.csv')
Load_info.head(5)

,bus,p_mw,q_mvar
Load_d,,,
1,1,0.00,0.00
2,2,0.10,0.06
3,3,0.09,0.04
4,4,0.12,0.08
5,5,0.06,0.03


In [19]:
"""
Y matrix
"""
ymat = m.makeYbus(mpc)

Y_mat_matpower = pd.DataFrame(ymat.todense())

Y_mat_matpower.index = range(1,ymat.shape[0]+1)
Y_mat_matpower.columns = range(1,ymat.shape[1]+1)
Y_mat_matpower.to_csv('./Pre_cal_data/Ymat_matpower.csv')

bus_multi_index = pd.MultiIndex.from_product(
    [range(1,ymat.shape[0]+1), range(1,ymat.shape[1]+1)],
    names=["Bus_i", "Bus_j"]
)

Y_mat_info = pd.DataFrame(index=bus_multi_index,columns=['Bus_G','Bus_B'])

for i in range(1,ymat.shape[0]+1):
    for j in range(1,ymat.shape[1]+1):
        Y_mat_info.loc[(i,j),'Bus_G'] = np.real(Y_mat_matpower.loc[i,j])
        Y_mat_info.loc[(i,j),'Bus_B'] = np.imag(Y_mat_matpower.loc[i,j])

Y_mat_info.to_csv('./Pre_cal_data/Y_mat_info.csv')        
Y_mat_info.head(5)  

Bus_G      Bus_B
Bus_i Bus_j                       
1     1      137.979749 -70.336748
      2     -137.979749  70.336748
      3             0.0        0.0
      4             0.0        0.0
      5             0.0        0.0

Matpower OPF 결과

In [20]:
# Run OPF
mpopt = m.mpoption('verbose', 2)
[baseMVA, bus, gen, gencost, branch, f, success, et] = m.runopf(mpc, mpopt, nout='max_nout')

mat_gen_index = range(1,len(gen)+1)
mat_gen_info_columns = ['bus','Pg',	'Qg','Qmax','Qmin','Vg','mBase','status','Pmax','Pmin','Pc1','Pc2','Qc1min','Qc1max','Qc2min','Qc2max','ramp_agc','ramp_10','ramp_30','ramp_q',	'apf','unknown1','unknown2','unknown3','unknown4']
mat_gen_info = pd.DataFrame(gen,index = mat_gen_index, columns = mat_gen_info_columns)

matpower_gen_mw_total = mat_gen_info['Pg'].sum() 

print('total gen MW:', matpower_gen_mw_total)


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0           100                    0.25         0.01           34            0
  1          74.3    0.41352   0.00341464    0.0312998      7.65954   0.00254455
  2     78.330235   0.034388  4.08164e-05   0.00361447     0.828056  0.000400051
  3     78.353541 0.00021783  3.32849e-09  0.000354744    0.0831788  2.31249e-06
  4     78.353543 1.1173e-08  9.13015e-15  6.87621e-09   0.00831789  1.13485e-10
  5     78.353543 4.4903e-15  9.20128e-15  1.47524e-15  0.000831789  1.72124e-18
  6     78.353543 3.9814e-15  9.82295e-15  7.45786e-16  8.31789e-05  3.44247e-18
  7     78.353543 3.5999e-15  5.33829e-15  1.10177e-15  8.31789e-06  1.72124e-18
  8     7